<a href="https://colab.research.google.com/github/FarhanDhanani/table_qa_datasets/blob/main/TABLE_Q_A_Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ENVIROMENT SETUP**


---



In [ ]:
# install datasets
%tensorflow_version 2.x
!pip install -U datasets
!pip install --upgrade langchain openai -q
#!pip install "pinecone-client[grpc]"
!pip install pinecone-client
!pip install transformers sentence_transformers torch-scatter
!pip install --upgrade protobuf
!pip install pandasql
!pip install --upgrade 'sqlalchemy<2.0'

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


# **IMPORTING LIBRARIES**


---


In [ ]:
from datasets import load_dataset
import json
import pprint
import pandas as pd
from pandasql import sqldf
from sqlalchemy import text

pp = pprint.PrettyPrinter(indent=4)

# **FeTaQA DATASET** (FREE FORM)


---

SOURCE = https://huggingface.co/datasets/DongfuTingle/FeTaQA

In [ ]:
datasetFeTaQA = load_dataset("DongfuTingle/FeTaQA")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
pp.pprint("************************* OVERVIEW OF DATASET *********************")
pp.pprint(datasetFeTaQA)

'************************* OVERVIEW OF DATASET *********************'
DatasetDict({
    train: Dataset({
        features: ['feta_id', 'table_source_json', 'page_wikipedia_url', 'table_page_title', 'table_section_title', 'table_array', 'highlighted_cell_ids', 'question', 'answer'],
        num_rows: 7326
    })
    validation: Dataset({
        features: ['feta_id', 'table_source_json', 'page_wikipedia_url', 'table_page_title', 'table_section_title', 'table_array', 'highlighted_cell_ids', 'question', 'answer'],
        num_rows: 1001
    })
    test: Dataset({
        features: ['feta_id', 'table_source_json', 'page_wikipedia_url', 'table_page_title', 'table_section_title', 'table_array', 'highlighted_cell_ids', 'question', 'answer'],
        num_rows: 2003
    })
})


In [ ]:
index =0
print("************************************** SAMPLE RECORD ****************************************")
print("QUESTION: "+ datasetFeTaQA['train'][0]['question'])
print("EXPECTED ANSWER: "+ datasetFeTaQA['train'][0]['answer'])
print("HIGHLIGHT CELLS: ", datasetFeTaQA['train'][0]['highlighted_cell_ids'])

print("\n************************* SAMPLE TABLE ***************************")
pd.DataFrame(datasetFeTaQA['train'][0]['table_array'], columns=datasetFeTaQA['train'][0]['table_array'][0])

************************************** SAMPLE RECORD ****************************************
QUESTION: Who won the 1982 Illinois gubernatorial election, and how many votes was the margin?
EXPECTED ANSWER: Thompson prevailed in the 1982 Illinois gubernatorial election by a 5,074 vote margin.
HIGHLIGHT CELLS:  [[1, 2], [6, 3]]

************************* SAMPLE TABLE ***************************


,Party,Party,Candidate,Votes,%,±
0,Party,Party,Candidate,Votes,%,±
1,-,Republican,James R. Thompson (incumbent),"1,816,101",49.44,-
2,-,Democratic,Adlai Stevenson III,"1,811,027",49.30,-
3,-,Libertarian,Bea Armstrong,"24,417",0.66,-
4,-,Taxpayers,John E. Roche,"22,001",0.60,-
5,-,N/A,write-ins,161,0.00,n-a
6,Majority,Majority,Majority,"5,074",0.14,-
7,Turnout,Turnout,Turnout,"3,673,707",-,-
8,-,Republican hold,Republican hold,Swing,-,-


In [ ]:
print("*********************************** FULL SAMPLE RECORD *************************************")
pp.pprint(datasetFeTaQA['train'][index])

*********************************** FULL SAMPLE RECORD *************************************
{   'answer': 'Thompson prevailed in the 1982 Illinois gubernatorial election '
              'by a 5,074 vote margin.',
    'feta_id': 18162,
    'highlighted_cell_ids': [[1, 2], [6, 3]],
    'page_wikipedia_url': 'http://en.wikipedia.org/wiki/1982_Illinois_gubernatorial_election',
    'question': 'Who won the 1982 Illinois gubernatorial election, and how '
                'many votes was the margin?',
    'table_array': [   ['Party', 'Party', 'Candidate', 'Votes', '%', '±'],
                       [   '-',
                           'Republican',
                           'James R. Thompson (incumbent)',
                           '1,816,101',
                           '49.44',
                           '-'],
                       [   '-',
                           'Democratic',
                           'Adlai Stevenson III',
                           '1,811,027',
                    

# **SQA DATASET** (FACTOID)


---

SOURCE = https://huggingface.co/datasets/msr_sqa/viewer/default/train?row=0


In [ ]:
datasetSQA = load_dataset("msr_sqa")

Generating train split:   0%|          | 0/12276 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2265 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3012 [00:00<?, ? examples/s]

In [ ]:
pp.pprint("************************* OVERVIEW OF DATASET *********************")
pp.pprint(datasetSQA)

'************************* OVERVIEW OF DATASET *********************'
DatasetDict({
    train: Dataset({
        features: ['id', 'annotator', 'position', 'question', 'question_and_history', 'table_file', 'table_header', 'table_data', 'answer_coordinates', 'answer_text'],
        num_rows: 12276
    })
    validation: Dataset({
        features: ['id', 'annotator', 'position', 'question', 'question_and_history', 'table_file', 'table_header', 'table_data', 'answer_coordinates', 'answer_text'],
        num_rows: 2265
    })
    test: Dataset({
        features: ['id', 'annotator', 'position', 'question', 'question_and_history', 'table_file', 'table_header', 'table_data', 'answer_coordinates', 'answer_text'],
        num_rows: 3012
    })
})


In [ ]:
index =0
print("************************************** SAMPLE RECORD ****************************************")
print("QUESTION: "+ datasetSQA['train'][index]['question'])
print("QUESTION HISTORY: ", datasetSQA['train'][index]['question_and_history'])
print("ANSWER TEXT:")
pp.pprint(datasetSQA['train'][index]['answer_text'])

print("\n************************* SAMPLE TABLE ***************************")
pd.DataFrame(datasetSQA['train'][index]['table_data'], columns=datasetSQA['train'][index]['table_header'])

************************************** SAMPLE RECORD ****************************************
QUESTION: where are the players from?
QUESTION HISTORY:  ['where are the players from?']
ANSWER TEXT:
[   'Louisiana State University',
    'Valley HS (Las Vegas, NV)',
    'Saugus (CA) HS',
    'Simeon HS (Chicago, IL)',
    'Texas Tech University',
    'Frankston (TX) HS',
    'Auburn University',
    'Lancaster (SC) HS',
    'Long Beach State University',
    'Westwood HS (Fort Pierce, FL)',
    'W.T. White High School (Dallas, TX)',
    'Salem (MA) HS',
    'Cal State Fullerton',
    'Fresno State University',
    'Hillsborough HS (Tampa, FL)',
    'Sarasota (FL) HS',
    'University of Iowa',
    'Jones County HS (Gray, GA)',
    'Fresno State University',
    'University of Texas',
    'Santa Clara University',
    'Fresno State University',
    'Seton Hall University',
    'University of Arizona',
    'Texas A&M University',
    'Hamden (CT) HS']

************************* SAMPLE TABLE 

,Pick,Player,Team,Position,School
0,1,Ben McDonald,Baltimore Orioles,RHP,Louisiana State University
1,2,Tyler Houston,Atlanta Braves,C,"Valley HS (Las Vegas, NV)"
2,3,Roger Salkeld,Seattle Mariners,RHP,Saugus (CA) HS
3,4,Jeff Jackson,Philadelphia Phillies,OF,"Simeon HS (Chicago, IL)"
4,5,Donald Harris,Texas Rangers,OF,Texas Tech University
5,6,Paul Coleman,Saint Louis Cardinals,OF,Frankston (TX) HS
6,7,Frank Thomas,Chicago White Sox,1B,Auburn University
7,8,Earl Cunningham,Chicago Cubs,OF,Lancaster (SC) HS
8,9,Kyle Abbott,California Angels,LHP,Long Beach State University
9,10,Charles Johnson,Montreal Expos,C,"Westwood HS (Fort Pierce, FL)"


In [ ]:
print("*********************************** FULL SAMPLE RECORD *************************************")
pp.pprint(datasetSQA['train'][index])

*********************************** FULL SAMPLE RECORD *************************************
{   'annotator': 0,
    'answer_coordinates': {   'column_index': [   4,
                                                  4,
                                                  4,
                                                  4,
                                                  4,
                                                  4,
                                                  4,
                                                  4,
                                                  4,
                                                  4,
                                                  4,
                                                  4,
                                                  4,
                                                  4,
                                                  4,
                                                  4,
                                       


# **WTQ** (FACTOID)
---
SOURCE: https://huggingface.co/datasets/TableQAKit/WTQ


In [ ]:
datasetWTQ = load_dataset("TableQAKit/WTQ")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
pp.pprint("************************* OVERVIEW OF DATASET *********************")
pp.pprint(datasetWTQ)

'************************* OVERVIEW OF DATASET *********************'
DatasetDict({
    train: Dataset({
        features: ['struct_in', 'id', 'question', 'text_in', 'answer_text', 'seq_out', 'table', 'table_id'],
        num_rows: 11321
    })
    test: Dataset({
        features: ['struct_in', 'id', 'question', 'text_in', 'answer_text', 'seq_out', 'table', 'table_id'],
        num_rows: 7175
    })
})


In [ ]:
index =0
print("************************************** SAMPLE RECORD ****************************************")
print("QUESTION: "+ datasetWTQ['train'][index]['question'])
print("ANSWER TEXT:", datasetWTQ['train'][index]['seq_out'])
pp.pprint(datasetWTQ['train'][index]['answer_text'])

print("\n************************* SAMPLE TABLE ***************************")
pd.DataFrame(datasetWTQ['train'][index]['table']['rows'], columns=datasetWTQ['train'][index]['table']['header'])

************************************** SAMPLE RECORD ****************************************
QUESTION: what was the last year where this team was a part of the usl a-league?
ANSWER TEXT: 2004
['2004']

************************* SAMPLE TABLE ***************************


,Year,Division,League,Regular Season,Playoffs,Open Cup,Avg. Attendance
0,2001,2,USL A-League,"4th, Western",Quarterfinals,Did not qualify,"7,169"
1,2002,2,USL A-League,"2nd, Pacific",1st Round,Did not qualify,"6,260"
2,2003,2,USL A-League,"3rd, Pacific",Did not qualify,Did not qualify,"5,871"
3,2004,2,USL A-League,"1st, Western",Quarterfinals,4th Round,"5,628"
4,2005,2,USL First Division,5th,Quarterfinals,4th Round,"6,028"
5,2006,2,USL First Division,11th,Did not qualify,3rd Round,"5,575"
6,2007,2,USL First Division,2nd,Semifinals,2nd Round,"6,851"
7,2008,2,USL First Division,11th,Did not qualify,1st Round,"8,567"
8,2009,2,USL First Division,1st,Semifinals,3rd Round,"9,734"
9,2010,2,USSF D-2 Pro League,"3rd, USL (3rd)",Quarterfinals,3rd Round,"10,727"


In [ ]:
print("*********************************** FULL SAMPLE RECORD *************************************")
pp.pprint(datasetWTQ['train'][index])

*********************************** FULL SAMPLE RECORD *************************************
{   'answer_text': ['2004'],
    'id': 'nt-0',
    'question': 'what was the last year where this team was a part of the usl '
                'a-league?',
    'seq_out': '2004',
    'struct_in': 'col : year | division | league | regular season | playoffs | '
                 'open cup | avg. attendance row 1 : 2001 | 2 | usl a-league | '
                 '4th, western | quarterfinals | did not qualify | 7,169 row 2 '
                 ': 2002 | 2 | usl a-league | 2nd, pacific | 1st round | did '
                 'not qualify | 6,260 row 3 : 2003 | 2 | usl a-league | 3rd, '
                 'pacific | did not qualify | did not qualify | 5,871 row 4 : '
                 '2004 | 2 | usl a-league | 1st, western | quarterfinals | 4th '
                 'round | 5,628 row 5 : 2005 | 2 | usl first division | 5th | '
                 'quarterfinals | 4th round | 6,028 row 6 : 2006 | 2 | usl '
         

# **WIKI SQL** (FACTOID)


---

SOURCE: https://huggingface.co/datasets/wikisql

In [ ]:
datasetWikiSQL = load_dataset("wikisql")

Generating test split:   0%|          | 0/15878 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8421 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/56355 [00:00<?, ? examples/s]

In [ ]:
pp.pprint("************************* OVERVIEW OF DATASET *********************")
pp.pprint(datasetWikiSQL)

'************************* OVERVIEW OF DATASET *********************'
DatasetDict({
    test: Dataset({
        features: ['phase', 'question', 'table', 'sql'],
        num_rows: 15878
    })
    validation: Dataset({
        features: ['phase', 'question', 'table', 'sql'],
        num_rows: 8421
    })
    train: Dataset({
        features: ['phase', 'question', 'table', 'sql'],
        num_rows: 56355
    })
})


In [ ]:
index =0
print("************************************** SAMPLE RECORD ****************************************")
print("QUESTION: "+ datasetWikiSQL['train'][index]['question'])

query = datasetWikiSQL['train'][index]['sql']['human_readable']
print("ORIGNAL SQL:", query)

headers = datasetWikiSQL['train'][index]['table']['header']
#mod_headers = [h.replace(' ', '') for h in orignal_headers]
tableInstance = pd.DataFrame(datasetWikiSQL['train'][index]['table']['rows'],
                             columns=headers)


print("\n************************* SAMPLE TABLE ***************************")
tableInstance

************************************** SAMPLE RECORD ****************************************
QUESTION: Tell me what the notes are for South Australia 
ORIGNAL SQL: SELECT Notes FROM table WHERE Current slogan = SOUTH AUSTRALIA

************************* SAMPLE TABLE ***************************


,State/territory,Text/background colour,Format,Current slogan,Current series,Notes
0,Australian Capital Territory,blue/white,Yaa·nna,ACT · CELEBRATION OF A CENTURY 2013,YIL·00A,Slogan screenprinted on plate
1,New South Wales,black/yellow,aa·nn·aa,NEW SOUTH WALES,BX·99·HI,No slogan on current series
2,New South Wales,black/white,aaa·nna,NSW,CPX·12A,Optional white slimline series
3,Northern Territory,ochre/white,Ca·nn·aa,NT · OUTBACK AUSTRALIA,CB·06·ZZ,New series began in June 2011
4,Queensland,maroon/white,nnn·aaa,QUEENSLAND · SUNSHINE STATE,999·TLG,Slogan embossed on plate
5,South Australia,black/white,Snnn·aaa,SOUTH AUSTRALIA,S000·AZD,No slogan on current series
6,Victoria,blue/white,aaa·nnn,VICTORIA - THE PLACE TO BE,ZZZ·562,Current series will be exhausted this year


In [ ]:
remove_ops = ["COUNT"]
conds = datasetWikiSQL['train'][index]['sql']['conds']['condition']
query = query if 'tableInstance' in query else query.replace('table', 'tableInstance')

for o_h in headers:
  if (o_h in query and (not (f"\"{o_h}\"" in query))):
    query = query.replace(o_h, f"\"{o_h}\"")

for c in conds:
  if (c in query and (not (f"'{c}'" in query))):
    query = query.replace(c,f"'{c}'")

for r in remove_ops:
  if (r in query):
    query = query.replace(r,"")

print("MODIFIED SQL:", query)
result = sqldf(query)
print("ANSWER: ")
result

MODIFIED SQL: SELECT "Notes" FROM tableInstance WHERE "Current slogan" = 'SOUTH AUSTRALIA'
ANSWER: 


,Notes
0,No slogan on current series


In [ ]:
print("*********************************** FULL SAMPLE RECORD *************************************")
pp.pprint(datasetWikiSQL['train'][index])

*********************************** FULL SAMPLE RECORD *************************************
{   'phase': 1,
    'question': 'Tell me what the notes are for South Australia ',
    'sql': {   'agg': 0,
               'conds': {   'column_index': [3],
                            'condition': ['SOUTH AUSTRALIA'],
                            'operator_index': [0]},
               'human_readable': 'SELECT Notes FROM table WHERE Current slogan '
                                 '= SOUTH AUSTRALIA',
               'sel': 5},
    'table': {   'caption': '',
                 'header': [   'State/territory',
                               'Text/background colour',
                               'Format',
                               'Current slogan',
                               'Current series',
                               'Notes'],
                 'id': '1-1000181-1',
                 'name': 'table_1000181_1',
                 'page_id': '',
                 'page_title': '',
       

# **HiTAB** (FACTOID)


---

SOURCE: https://huggingface.co/datasets/kasnerz/hitab

In [ ]:
datasetHiTab = load_dataset("kasnerz/hitab")

In [ ]:
pp.pprint("************************* OVERVIEW OF DATASET *********************")
pp.pprint(datasetHiTab)

'************************* OVERVIEW OF DATASET *********************'
DatasetDict({
    train: Dataset({
        features: ['id', 'table_id', 'table_source', 'sentence_id', 'sub_sentence_id', 'sub_sentence', 'question', 'answer', 'aggregation', 'linked_cells', 'answer_formulas', 'reference_cells_map', 'table_content'],
        num_rows: 7417
    })
    validation: Dataset({
        features: ['id', 'table_id', 'table_source', 'sentence_id', 'sub_sentence_id', 'sub_sentence', 'question', 'answer', 'aggregation', 'linked_cells', 'answer_formulas', 'reference_cells_map', 'table_content'],
        num_rows: 1671
    })
    test: Dataset({
        features: ['id', 'table_id', 'table_source', 'sentence_id', 'sub_sentence_id', 'sub_sentence', 'question', 'answer', 'aggregation', 'linked_cells', 'answer_formulas', 'reference_cells_map', 'table_content'],
        num_rows: 1584
    })
})


In [ ]:
index =1
print("************************************** SAMPLE RECORD ****************************************")
print("QUESTION: "+ datasetHiTab['train'][index]['question'])
print("ANSWER: "+ datasetHiTab['train'][index]['answer'])


table_contents = datasetHiTab['train'][index]['table_content']
table = json.loads(table_contents.replace("'", "\""))

# processing col headers.
columns = []
header_rows = table['top_header_rows_num']
for row in range(1, header_rows):
  headers = table['texts'][row-1]
  p = ''
  mod_headers = [p:=h if (h!=None and h!='') else p for h in headers]
  columns.append(mod_headers)
columnsMI = pd.MultiIndex.from_arrays(columns)

tableInstance = pd.DataFrame(table['texts'][header_rows-1:], columns=columnsMI)

print("\n************************* SAMPLE TABLE ***************************")
tableInstance

************************************** SAMPLE RECORD ****************************************
QUESTION: how many games did schwartz, the manager of rot-weiß erfurt between 11 april 2003 and 30 june 2003, won out of 10 matches?
ANSWER: [1.0]

************************* SAMPLE TABLE ***************************


team               from                 to record       \
                                                                      g    w   
0          rot-weiß erfurt      11 april 2003       30 june 2003     10    1   
1  1. fc kaiserslautern ii     1 january 2007   9 september 2012    193   86   
2     1. fc kaiserslautern         4 may 2009       30 june 2009      4    1   
3          rot-weiß erfurt  10 september 2012  30 september 2013     30   10   
4            sv sandhausen        1 july 2013       25 june 2016    108   35   
5           1. fc nurnberg       25 june 2016       7 march 2017     25    8   
6            karlsruher sc     29 august 2017            present     46   23   
7                    total                                          416  164   

                          
     d    l  win %   ref  
0    3    6  10.00  none  
1   48   59  44.56  none  
2    0    3  25.00  none  
3    9   11  33.33  none  
4   30   43  32.41  none  
5    6   11  32.00  none  
6   15    8  50.00  none  
7  111  141  39.42     -

In [ ]:
print("*********************************** FULL SAMPLE RECORD *************************************")
pp.pprint(datasetHiTab['train'][index])

*********************************** FULL SAMPLE RECORD *************************************
{   'aggregation': "['none']",
    'answer': '[1.0]',
    'answer_formulas': "['=E5']",
    'id': '249b7a7903f9dec2873b2ff4eafb8075',
    'linked_cells': "{'entity_link': {'top': {'matches': {'(1, 3)': 'g'}, "
                    "'won': {'(1, 4)': 'w'}}, 'left': {'rot-weiß erfurt': "
                    "{'(2, 0)': 'rot-weiß erfurt'}, '11 april 2003': {'(2, "
                    "1)': '11 april 2003'}, '30 june 2003': {'(2, 2)': '30 "
                    "june 2003'}}, 'top_left_corner': {}}, 'quantity_link': "
                    "{'10': {'(2, 3)': 10.0}, '[ANSWER]': {'(2, 4)': 1.0}}}",
    'question': 'how many games did schwartz, the manager of rot-weiß erfurt '
                'between 11 april 2003 and 30 june 2003, won out of 10 '
                'matches?',
    'reference_cells_map': "{'E5': '(2, 4)'}",
    'sentence_id': '0',
    'sub_sentence': 'schwartz was the manager of rot-weiß er

#FINQA (FACTOID+PRE/POST ANOTATIONS)


---

SOURCE: https://huggingface.co/datasets/dreamerdeo/finqa

In [ ]:
datasetFinQA = load_dataset("dreamerdeo/finqa")

Repo card metadata block was not found. Setting CardData to empty.


/root/.cache/huggingface/datasets/downloads/extracted/ca045c27488b49db24426c67e0b35e06b0188746c40ffaf8d01a7b51e52924dc


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
pp.pprint("************************* OVERVIEW OF DATASET *********************")
pp.pprint(datasetFinQA)

'************************* OVERVIEW OF DATASET *********************'
DatasetDict({
    train: Dataset({
        features: ['id', 'post_text', 'pre_text', 'question', 'answer', 'gold_evidence', 'table'],
        num_rows: 6251
    })
    validation: Dataset({
        features: ['id', 'post_text', 'pre_text', 'question', 'answer', 'gold_evidence', 'table'],
        num_rows: 883
    })
    test: Dataset({
        features: ['id', 'post_text', 'pre_text', 'question', 'answer', 'gold_evidence', 'table'],
        num_rows: 1147
    })
})


In [ ]:
index =0
print("************************************** SAMPLE RECORD ****************************************")
print("QUESTION: "+ datasetFinQA['train'][index]['question'])
print("---------------------------PRE-TEXT------------------------------------")
pp.pprint(datasetFinQA['train'][index]['pre_text'])
print("--------------------------POST-TEXT------------------------------------")
pp.pprint(datasetFinQA['train'][index]['post_text'])
print("-----------------------------------------------------------------------")
print("ANSWER: "+ datasetFinQA['train'][index]['answer'])
print("---------------------------GOLD EVIDENCE-------------------------------")
pp.pprint(datasetFinQA['train'][index]['gold_evidence'])



tableInstance = pd.DataFrame(datasetFinQA['train'][index]['table'][1:],
                             columns = datasetFinQA['train'][index]['table'][0])

print("\n************************* SAMPLE TABLE ***************************")
tableInstance

************************************** SAMPLE RECORD ****************************************
QUESTION: what is the the interest expense in 2009?
---------------------------PRE-TEXT------------------------------------
[   'interest rate to a variable interest rate based on the three-month libor '
    'plus 2.05% ( 2.05 % ) ( 2.34% ( 2.34 % ) as of october 31 , 2009 ) .',
    'if libor changes by 100 basis points , our annual interest expense would '
    'change by $ 3.8 million .',
    'foreign currency exposure as more fully described in note 2i .',
    'in the notes to consolidated financial statements contained in item 8 of '
    'this annual report on form 10-k , we regularly hedge our non-u.s .',
    'dollar-based exposures by entering into forward foreign currency exchange '
    'contracts .',
    'the terms of these contracts are for periods matching the duration of the '
    'underlying exposure and generally range from one month to twelve months .',
    'currently , our larges

,,october 31 2009,november 1 2008
0,fair value of forward exchange contracts asset...,$ 6427,$ -23158 ( 23158 )
1,fair value of forward exchange contracts after...,$ 20132,$ -9457 ( 9457 )
2,fair value of forward exchange contracts after...,$ -6781 ( 6781 ),$ -38294 ( 38294 )


In [ ]:
print("*********************************** FULL SAMPLE RECORD *************************************")
pp.pprint(datasetFinQA['train'][index])

*********************************** FULL SAMPLE RECORD *************************************
{   'answer': '380',
    'gold_evidence': [   'if libor changes by 100 basis points , our annual '
                         'interest expense would change by $ 3.8 million .'],
    'id': 'ADI/2009/page_49.pdf-1',
    'post_text': [   'fair value of forward exchange contracts after a 10% ( '
                     '10 % ) unfavorable movement in foreign currency exchange '
                     'rates asset ( liability ) .',
                     '.',
                     '.',
                     '.',
                     '.',
                     '.',
                     '.',
                     '.',
                     '.',
                     '$ 20132 $ ( 9457 ) fair value of forward exchange '
                     'contracts after a 10% ( 10 % ) favorable movement in '
                     'foreign currency exchange rates liability .',
                     '.',
                     '.',
   

# LIST OF POPULAR TABLE Q/A DATASETS NOT LISTED HERE



---
- TATQA
- Hybrid QA
- SPYDER
